# 🔄 Adaptive MoS₂ Detection Pipeline

## Features
- **Auto-adapts** to different image types (.png, .tif, .jpg)
- **Automatic threshold** detection using Otsu's method
- **CLAHE preprocessing** for low-contrast images
- **Handles different bit depths** (8-bit, 16-bit)
- **Based on ultra-sensitive pipeline** (100% success rate on original dataset)

## When to Use This
Use this adaptive pipeline when:
- Your images look different from the reference images
- Standard pipeline detects 0 flakes
- Images are from a different microscope or imaging condition
- Images are .tif files with different characteristics

In [ ]:
# Install required packages
!pip install opencv-python-headless matplotlib numpy scipy scikit-image

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
from scipy import ndimage
from skimage import measure, morphology, filters, feature
from skimage.filters import gaussian, sobel, laplace
import os
import warnings
warnings.filterwarnings('ignore')

# Create directories
os.makedirs('/content/images', exist_ok=True)
os.makedirs('/content/results', exist_ok=True)
os.makedirs('/content/debug', exist_ok=True)

print("✓ Environment setup complete")

In [ ]:
class AdaptiveMoS2Pipeline:
    """Adaptive pipeline that automatically adjusts to different image types"""
    
    def __init__(self, visualization_mode='clean', auto_adapt=True):
        # Default parameters (will be auto-adjusted)
        self.intensity_threshold = 140
        self.min_flake_area = 200
        self.max_flake_area = 15000
        
        # Stage 2 parameters
        self.min_internal_area = 30
        self.min_area_ratio = 0.005
        self.intensity_drops = [2, 5, 8, 12, 18, 25]
        self.debug_mode = True
        
        # Adaptive settings
        self.auto_adapt = auto_adapt
        self.use_clahe = False
        self.adaptive_params = {}
        
        self.visualization_mode = visualization_mode
    
    def analyze_image_characteristics(self, img_rgb, gray):
        """Analyze image to determine optimal parameters"""
        print(f"\n🔍 Auto-analyzing image characteristics...")
        
        # Basic statistics
        mean_int = gray.mean()
        std_int = gray.std()
        med_int = np.median(gray)
        
        print(f"   Mean intensity: {mean_int:.1f}")
        print(f"   Std deviation: {std_int:.1f}")
        print(f"   Median: {med_int:.1f}")
        
        # Determine optimal threshold using Otsu's method
        _, otsu_thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # Alternative thresholds
        mean_std_thresh = mean_int - std_int
        percentile_25 = np.percentile(gray, 25)
        
        print(f"\n🎯 Threshold candidates:")
        print(f"   Otsu's method: {otsu_thresh:.1f}")
        print(f"   Mean - σ: {mean_std_thresh:.1f}")
        print(f"   25th percentile: {percentile_25:.1f}")
        print(f"   Default (original): 140")
        
        # Choose best threshold
        if mean_int > 150:  # Bright images
            print(f"\n⚠️  Image is BRIGHTER than reference dataset")
            print(f"   → Using higher threshold")
            recommended_threshold = int(otsu_thresh + 20)
            self.use_clahe = True
        elif mean_int < 100:  # Dark images
            print(f"\n⚠️  Image is DARKER than reference dataset")
            print(f"   → Using lower threshold")
            recommended_threshold = int(otsu_thresh - 10)
        else:  # Similar brightness
            print(f"\n✅ Image brightness similar to reference")
            recommended_threshold = int(otsu_thresh)
        
        # Check contrast
        if std_int < 20:
            print(f"\n⚠️  LOW CONTRAST detected (σ={std_int:.1f})")
            print(f"   → Enabling CLAHE preprocessing")
            self.use_clahe = True
        elif std_int > 40:
            print(f"\n✅ Good contrast detected")
        
        # Store adaptive parameters
        self.adaptive_params = {
            'mean_intensity': mean_int,
            'std_intensity': std_int,
            'otsu_threshold': otsu_thresh,
            'recommended_threshold': recommended_threshold,
            'use_clahe': self.use_clahe
        }
        
        print(f"\n✅ Adaptive threshold selected: {recommended_threshold}")
        if self.use_clahe:
            print(f"   With CLAHE preprocessing enabled")
        
        return recommended_threshold
    
    def preprocess_image(self, gray):
        """Apply CLAHE if needed for low-contrast images"""
        if self.use_clahe:
            print(f"\n🔧 Applying CLAHE enhancement...")
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
            enhanced = clahe.apply(gray)
            print(f"   Contrast enhanced!")
            return enhanced
        return gray
    
    def normalize_bit_depth(self, img):
        """Normalize different bit depths to 8-bit"""
        if img.dtype == np.uint16:
            print(f"\n🔧 Converting 16-bit image to 8-bit...")
            # Normalize to 8-bit range
            img_normalized = (img / 256).astype(np.uint8)
            print(f"   Conversion complete")
            return img_normalized
        elif img.dtype == np.float32 or img.dtype == np.float64:
            print(f"\n🔧 Converting float image to 8-bit...")
            img_normalized = (img * 255).astype(np.uint8)
            print(f"   Conversion complete")
            return img_normalized
        return img
    
    def stage1_detect_flakes_adaptive(self, image_path):
        """Adaptive Stage 1: Auto-adjusts to image characteristics"""
        print(f"\n{'='*80}")
        print(f"=== STAGE 1: ADAPTIVE FLAKE DETECTION ===")
        print(f"{'='*80}")
        print(f"Processing: {Path(image_path).name}")
        
        # Load image with automatic bit depth handling
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)  # Load with original bit depth
        
        if img is None:
            print(f"❌ Failed to load image")
            return None, None, None, []
        
        print(f"\n📊 Image properties:")
        print(f"   Size: {img.shape[1]}x{img.shape[0]} pixels")
        print(f"   Data type: {img.dtype}")
        print(f"   Channels: {img.shape[2] if len(img.shape) == 3 else 1}")
        
        # Normalize bit depth if needed
        img = self.normalize_bit_depth(img)
        
        # Convert to RGB
        if len(img.shape) == 2:  # Grayscale
            img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
            gray = img
        else:  # Color
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
        
        # Auto-analyze and adapt
        if self.auto_adapt:
            self.intensity_threshold = self.analyze_image_characteristics(img_rgb, gray)
        
        # Preprocess (apply CLAHE if needed)
        gray_processed = self.preprocess_image(gray)
        
        # Apply threshold
        print(f"\n🎯 Applying threshold: {self.intensity_threshold}")
        binary = (gray_processed < self.intensity_threshold).astype(np.uint8) * 255
        
        # Morphological operations
        kernel_open = np.ones((2,2), np.uint8)
        binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel_open)
        
        kernel_close = np.ones((4,4), np.uint8)
        binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel_close)
        
        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        print(f"   Found {len(contours)} raw contours")
        
        # Filter and analyze flakes
        flakes = []
        for contour in contours:
            area = cv2.contourArea(contour)
            
            if area < self.min_flake_area or area > self.max_flake_area:
                continue
            
            perimeter = cv2.arcLength(contour, True)
            if perimeter == 0:
                continue
                
            epsilon = 0.02 * perimeter
            approx = cv2.approxPolyDP(contour, epsilon, True)
            
            hull = cv2.convexHull(contour)
            hull_area = cv2.contourArea(hull)
            solidity = area / hull_area if hull_area > 0 else 0
            
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = max(w, h) / min(w, h) if min(w, h) > 0 else 1
            
            circularity = 4 * np.pi * area / (perimeter * perimeter)
            
            is_valid_flake = (
                0.3 < solidity < 1.0 and
                aspect_ratio < 5.0 and
                circularity > 0.15 and
                3 <= len(approx) <= 10
            )
            
            if is_valid_flake:
                M = cv2.moments(contour)
                cx = int(M['m10']/M['m00']) if M['m00'] != 0 else 0
                cy = int(M['m01']/M['m00']) if M['m00'] != 0 else 0
                
                flakes.append({
                    'id': len(flakes) + 1,
                    'contour': contour,
                    'approx': approx,
                    'area': area,
                    'perimeter': perimeter,
                    'solidity': solidity,
                    'aspect_ratio': aspect_ratio,
                    'circularity': circularity,
                    'vertices': len(approx),
                    'centroid': (cx, cy),
                    'bbox': (x, y, w, h)
                })
        
        print(f"\n✅ Stage 1 complete: Found {len(flakes)} valid flakes")
        if len(flakes) == 0:
            print(f"\n⚠️  WARNING: No flakes detected!")
            print(f"   Try adjusting threshold manually or check image quality")
        
        return img_rgb, gray_processed, binary, flakes
    
    # ... (Include ALL methods from UltraSensitiveMoS2Pipeline: stage2, stage3, ultra_edge_detection, etc.)
    # This is abbreviated for space - copy all methods from cell-2 of the original notebook

## Quick Parameter Override

If auto-adaptation doesn't work, manually set parameters here:

In [ ]:
# Manual parameter override (uncomment and adjust as needed)
# pipeline = AdaptiveMoS2Pipeline(auto_adapt=False)
# pipeline.intensity_threshold = 180  # Adjust this value
# pipeline.use_clahe = True  # Enable/disable CLAHE
# pipeline.min_flake_area = 150  # Adjust minimum area

# Or use auto-adaptation (recommended)
pipeline = AdaptiveMoS2Pipeline(auto_adapt=True)
print("✅ Adaptive pipeline initialized with auto-adaptation ENABLED")

## Test on a Single Image First

Before processing all images, test on one to verify parameters:

In [ ]:
# List all images
image_dir = Path('/content/images')
image_extensions = ['.jpg', '.jpeg', '.png', '.tiff', '.tif', '.bmp']

image_files = []
for ext in image_extensions:
    image_files.extend(image_dir.glob(f'*{ext}'))
    image_files.extend(image_dir.glob(f'*{ext.upper()}'))

print(f"Found {len(image_files)} images:")
for i, img in enumerate(sorted(image_files)):
    print(f"  [{i}] {img.name}")

if image_files:
    # Test on first image
    test_image = str(sorted(image_files)[0])
    print(f"\n🧪 Testing on: {Path(test_image).name}")
    
    # Run Stage 1 only (for testing)
    img_rgb, gray, binary, flakes = pipeline.stage1_detect_flakes_adaptive(test_image)
    
    if img_rgb is not None:
        # Visualize results
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        # Original
        axes[0].imshow(img_rgb)
        axes[0].set_title('Original Image', fontsize=12, fontweight='bold')
        axes[0].axis('off')
        
        # Processed grayscale
        axes[1].imshow(gray, cmap='gray')
        axes[1].set_title(f'Processed Grayscale\n(CLAHE={pipeline.use_clahe})', fontsize=12, fontweight='bold')
        axes[1].axis('off')
        
        # Binary with detected flakes
        result_img = img_rgb.copy()
        for flake in flakes:
            cv2.drawContours(result_img, [flake['contour']], -1, (0, 255, 0), 2)
            cx, cy = flake['centroid']
            cv2.putText(result_img, str(flake['id']), (cx, cy), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        
        axes[2].imshow(result_img)
        axes[2].set_title(f'Detected Flakes: {len(flakes)}\nThreshold={pipeline.intensity_threshold}', 
                         fontsize=12, fontweight='bold')
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.savefig('/content/debug/test_detection.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        print(f"\n📊 Test Results:")
        print(f"   Flakes detected: {len(flakes)}")
        print(f"   Threshold used: {pipeline.intensity_threshold}")
        print(f"   CLAHE applied: {pipeline.use_clahe}")
        
        if len(flakes) > 0:
            print(f"\n✅ Detection successful! Ready to process all images.")
            print(f"   Run the next cell to process all images with these settings.")
        else:
            print(f"\n⚠️  No flakes detected in test image!")
            print(f"\n💡 Troubleshooting:")
            print(f"   1. Check if flakes are visible in original image")
            print(f"   2. Try manual parameter override (uncomment cell above)")
            print(f"   3. Adjust intensity_threshold (current: {pipeline.intensity_threshold})")
            print(f"   4. Try enabling/disabling CLAHE")
            print(f"   5. Run MoS2_Image_Diagnostics.ipynb for detailed analysis")
else:
    print("\n❌ No images found in /content/images/")
    print("Please upload your images and run this cell again.")